In [1]:
import numpy as np
import pandas as pd
import os

import requests
from  bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep
import random

In [3]:
browser = webdriver.Chrome()

browser.get("https://nba.com/games")
browser.maximize_window()

# cookies
sleep(1)
browser.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()

# Ir a la página de equipos
sleep(2)
browser.find_element(by=By.XPATH, value='//*[@id="nav-ul"]/li[7]/a').click()


In [4]:
html_equipos = browser.page_source
soup = BeautifulSoup(html_equipos, "html.parser")
urls_equipos_nba = [a['href'] for a in soup.find_all('a', class_='Anchor_anchor__cSc3P TeamFigureLink_teamFigureLink__uqnNO') if '/stats/team/' in a['href']]
print(urls_equipos_nba)

['/stats/team/1610612738', '/stats/team/1610612751', '/stats/team/1610612752', '/stats/team/1610612755', '/stats/team/1610612761', '/stats/team/1610612741', '/stats/team/1610612739', '/stats/team/1610612765', '/stats/team/1610612754', '/stats/team/1610612749', '/stats/team/1610612737', '/stats/team/1610612766', '/stats/team/1610612748', '/stats/team/1610612753', '/stats/team/1610612764', '/stats/team/1610612743', '/stats/team/1610612750', '/stats/team/1610612760', '/stats/team/1610612757', '/stats/team/1610612762', '/stats/team/1610612744', '/stats/team/1610612746', '/stats/team/1610612747', '/stats/team/1610612756', '/stats/team/1610612758', '/stats/team/1610612742', '/stats/team/1610612745', '/stats/team/1610612763', '/stats/team/1610612740', '/stats/team/1610612759']


In [4]:
# Para acceder a las estadísticas del primer equipo
# NO EJECUTAR si queremos hacer directamente el bucle con todos
sleep(0.3)

browser.find_element(By.CSS_SELECTOR, '#__next > div.Layout_base__6IeUC.Layout_justNav__2H4H0 > div.Layout_mainContent__jXliI > div.MaxWidthContainer_mwc__ID5AG > section > div > div.TeamDivisions_wrapper__5_SVo > div:nth-child(1) > div.TeamDivisions_divisionTeams__N6vcW > div:nth-child(1) > div > div > a:nth-child(2)').click()


In [4]:
# NO EJECUTAR si queremos hacer el bucle con todos, esto solo hace una extracción que me sirvió de guía
# Igual que hicimos con la de jugador, sacamos la ID del equipo del html
url_actual = browser.current_url
id_equipo = url_actual.split('/')[-1]

# Obtener el nombre del equipo
nombre_equipo_elemento = browser.find_element(By.CSS_SELECTOR, ".TeamHeader_name__MmHlP")
nombre_equipo = nombre_equipo_elemento.text

# Esto es para obtener la racha de victorias y también la posición en la división
registro_elemento = browser.find_element(By.CSS_SELECTOR, ".TeamHeader_record__wzofp")
registro_datos = registro_elemento.text.split('|')
victorias_derrotas = registro_datos[0].strip()
puesto = registro_datos[1].split('in')[0].strip()
division = registro_datos[1].split('in')[1].strip()

# Esto es para obtener las estadísticas esas del cuadro superior derecho de la página de equipos
ppg_elemento = browser.find_element(By.CSS_SELECTOR, "div.TeamHeader_rank__lMnzF:nth-of-type(1) .TeamHeader_rankValue__ZGDCq")
ppg = ppg_elemento.text

rpg_elemento = browser.find_element(By.CSS_SELECTOR, "div.TeamHeader_rank__lMnzF:nth-of-type(2) .TeamHeader_rankValue__ZGDCq")
rpg = rpg_elemento.text

apg_elemento = browser.find_element(By.CSS_SELECTOR, "div.TeamHeader_rank__lMnzF:nth-of-type(3) .TeamHeader_rankValue__ZGDCq")
apg = apg_elemento.text

oppg_elemento = browser.find_element(By.CSS_SELECTOR, "div.TeamHeader_rank__lMnzF:nth-of-type(4) .TeamHeader_rankValue__ZGDCq")
oppg = oppg_elemento.text


print(f"ID del equipo: {id_equipo}")
print(f"Nombre del equipo: {nombre_equipo}")
print(f"Registro (Victorias-Derrotas): {victorias_derrotas}")
print(f"Puesto: {puesto}, División: {division}")
print(f"PPG: {ppg}, RPG: {rpg}, APG: {apg}, OPPG: {oppg}")


ID del equipo: 1610612738
Nombre del equipo: BOSTON 
CELTICS
Registro (Victorias-Derrotas): 28 - 7
Puesto: 1st, División: Eastern
PPG: 120.9, RPG: 47.8, APG: 26.0, OPPG: 110.1


In [5]:
def extraer_informacion_equipo_bs4(url):
    try:        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        id_equipo = url.split('/')[-1]
        
        nombre_equipo_elemento = soup.select_one(".TeamHeader_name__MmHlP")
        nombre_equipo = nombre_equipo_elemento.text if nombre_equipo_elemento else "none"

        registro_elemento = soup.select_one(".TeamHeader_record__wzofp")
        if registro_elemento:
            registro_datos = registro_elemento.text.split('|')
            victorias_derrotas = registro_datos[0].strip()
            puesto = registro_datos[1].split('in')[0].strip()
            division = registro_datos[1].split('in')[1].strip()
        else:
            victorias_derrotas = puesto = division = "none"

        ppg = rpg = apg = oppg = "none"
        estadisticas_elementos = soup.select("div.TeamHeader_rank__lMnzF .TeamHeader_rankValue__ZGDCq")
        if estadisticas_elementos:
            ppg = estadisticas_elementos[0].text if len(estadisticas_elementos) > 0 else "none"
            rpg = estadisticas_elementos[1].text if len(estadisticas_elementos) > 1 else "none"
            apg = estadisticas_elementos[2].text if len(estadisticas_elementos) > 2 else "none"
            oppg = estadisticas_elementos[3].text if len(estadisticas_elementos) > 3 else "none"

        return {
            "id_equipo": id_equipo,
            "nombre_equipo": nombre_equipo,
            "victorias_derrotas": victorias_derrotas,
            "puesto": puesto,
            "division": division,
            "ppg": ppg,
            "rpg": rpg,
            "apg": apg,
            "oppg": oppg,
        }
    except Exception as e:
        print(f"Error al acceder: {e}")
        return None


In [6]:
id_equipo, nombre_equipo, victorias_derrotas, puesto, division, ppg, rpg, apg, oppg = [], [], [], [], [], [], [], [], []


In [7]:
url_base = "https://www.nba.com"
web_equipos_nba = [url_base + div.select("a")[1]["href"] for div in soup.find_all("div", class_="TeamFigure_tfLinks__gwWFj")]

for url in web_equipos_nba:
    print(f"Accediendo a: {url}")
    info_equipo = extraer_informacion_equipo_bs4(url)
    if info_equipo:
        id_equipo.append(info_equipo['id_equipo'])
        nombre_equipo.append(info_equipo['nombre_equipo'])
        victorias_derrotas.append(info_equipo['victorias_derrotas'])
        puesto.append(info_equipo['puesto'])
        division.append(info_equipo['division'])
        ppg.append(info_equipo['ppg'])
        rpg.append(info_equipo['rpg'])
        apg.append(info_equipo['apg'])
        oppg.append(info_equipo['oppg'])

    
    sleep(random.uniform(2, 4))


Accediendo a: https://www.nba.com/stats/team/1610612738
Accediendo a: https://www.nba.com/stats/team/1610612751
Accediendo a: https://www.nba.com/stats/team/1610612752
Accediendo a: https://www.nba.com/stats/team/1610612755
Accediendo a: https://www.nba.com/stats/team/1610612761
Accediendo a: https://www.nba.com/stats/team/1610612741
Accediendo a: https://www.nba.com/stats/team/1610612739
Accediendo a: https://www.nba.com/stats/team/1610612765
Accediendo a: https://www.nba.com/stats/team/1610612754
Accediendo a: https://www.nba.com/stats/team/1610612749
Accediendo a: https://www.nba.com/stats/team/1610612737
Accediendo a: https://www.nba.com/stats/team/1610612766
Accediendo a: https://www.nba.com/stats/team/1610612748
Accediendo a: https://www.nba.com/stats/team/1610612753
Accediendo a: https://www.nba.com/stats/team/1610612764
Accediendo a: https://www.nba.com/stats/team/1610612743
Accediendo a: https://www.nba.com/stats/team/1610612750
Accediendo a: https://www.nba.com/stats/team/161

In [8]:
df_equipos = pd.DataFrame({
    'ID Equipo': id_equipo,
    'Nombre Equipo': nombre_equipo,
    'Victorias-Derrotas': victorias_derrotas,
    'Puesto': puesto,
    'División': division,
    'PPG': ppg,
    'RPG': rpg,
    'APG': apg,
    'OPPG': oppg
})


In [9]:
print(df_equipos)

     ID Equipo           Nombre Equipo Victorias-Derrotas Puesto División  \
0   1610612738          Boston Celtics             28 - 8    1st  Eastern   
1   1610612751           Brooklyn Nets            16 - 21   10th  Eastern   
2   1610612752         New York Knicks            21 - 15    8th  Eastern   
3   1610612755      Philadelphia 76ers            23 - 12    3rd  Eastern   
4   1610612761         Toronto Raptors            15 - 21   11th  Eastern   
5   1610612741           Chicago Bulls            17 - 21    9th  Eastern   
6   1610612739     Cleveland Cavaliers            21 - 15    7th  Eastern   
7   1610612765         Detroit Pistons             3 - 33   15th  Eastern   
8   1610612754          Indiana Pacers            21 - 15    6th  Eastern   
9   1610612749         Milwaukee Bucks            25 - 12    2nd  Eastern   
10  1610612737           Atlanta Hawks            14 - 21   12th  Eastern   
11  1610612766       Charlotte Hornets             8 - 26   13th  Eastern   

In [9]:
df_equipos.to_excel("excels/equipos_nba.xlsx", index=False, engine='openpyxl')


In [10]:
# Hacemos el guardado que estamos ya realizando para si existse el archivo, que no se sobreescriba

def guardar_excel_con_numeracion(df, nombre_base):
    directorio = os.path.join(os.getcwd(), "excels")

    if not os.path.exists(directorio):
        os.makedirs(directorio)

    ruta_base = os.path.join(directorio, nombre_base)

    contador = 0
    ruta_final = f"{ruta_base}.xlsx"

    while os.path.exists(ruta_final):
        contador += 1
        ruta_final = f"{ruta_base}_{contador}.xlsx"

    df.to_excel(ruta_final, index=False, engine='openpyxl')
    print(f"Archivo guardado como: {ruta_final}")


guardar_excel_con_numeracion(df_equipos, "equipos_nba")

Archivo guardado como: c:\Users\34670\Desktop\python\Hack a boss\proyecto_1\excels\equipos_nba_1.xlsx
